# Tensor

In [2]:
import numpy as np
import torch

## Tensor简介
**张量其实是一个多维数组，它是标量、向量、矩阵的高维拓展**

data: 被包装的Tensor

grad: data的梯度

grad_fn: fn表示function的意思，记录我么创建的创建张量时用到的方法，比如说加法，乘法，这个操作在求导过程需要用到，Tensor的Function， 是自动求导的关键

requires_grad: 指示是否需要梯度， 有的不需要梯度

is_leaf: 指示是否是叶子节点（张量）

dtype: 张量的数据类型， 如torch.FloatTensor, torch.cuda.FloatTensor, 用的最多的一般是float32和int64(torch.long)

shape: 张量的形状， 如(64, 3, 224, 224)

device: 张量所在的设备， GPU/CPU， 张量放在GPU上才能使用加速。

## Tensor创建

In [3]:
# from data(list or numpy)
arr=np.ones((3,3))
t1=torch.tensor(arr)
print(t1)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)


In [ ]:
# share memory of np and tensor
arr = np.array([[1, 2, 3], [4, 5, 6]])
t2 = torch.from_numpy(arr)

# create from numbers
t3=torch.zeros((3,3))

# create zeros_tensor as tensor's size
t4=torch.zeros_like(t3)

# all-X tensor can be created by this
t5=torch.full((4,4),2)

# 创建等差的1维张量，数值区间[start, end)
t6 = torch.arange(2, 10, 2)  # tensor([2, 4, 6, 8])

# 创建均分的1维张量， 数值区间[start, end] 注意这里都是闭区间
t = torch.linspace(2, 10, 5)   # tensor([2, 4, 6, 8, 10])

# 那么如果不是那么正好呢？ 步长应该是多少？
t = torch.linspace(2, 10, 6)   # tensor([2, 3.6, 5.2, 6.8, 8.4, 10])

# 创建对数均分数列：
t=torch.logspace(1,100,steps=20,base=2)

# 创建单位对角矩阵， 默认是方阵
t=torch.eye(10)

# 生成正态分布（高斯分布）,mean是均值，std是标准差。
#根据mean和std，分别各有两种取值，所以这里会有四种模式:（标量，张量）
t=torch.normal(0,1,size=(4,))

# 生成均匀分布：torch.rand(), rand_like() 在[0,1)生成均匀分布
# torch.randint(), torch.randint_like(): 区间[low,hight)生成整数均匀分布
# torch.randperm(n): 生成从0 - n-1的随机排列, n是张量的长度, 经常用来生成一个乱序索引。
# torch.bernoulli(input): 以input为概率，生成伯努利分布(0-1分布，两点分布）， input: 概率值


## Tensor操作

**张量的拼接**

torch.cat(tensors, dim=0, out=None): 将张量按维度dim进行拼接, tensors表示张量序列， dim要拼接的维度

torch.stack(tensors, dim=0, out=None): 在新创建的维度dim上进行拼接， tensors表示张量序列， dim要拼接的维度


In [ ]:
# 张量的拼接
t = torch.ones((2, 3))
print(t)

t_0 = torch.cat([t, t], dim=0)       # 行拼接
t_1 = torch.cat([t, t], dim=1)    # 列拼接
print(t_0, t_0.shape)
print(t_1, t_1.shape)

# 结果：
# tensor([[1., 1., 1.],
#     [1., 1., 1.]])
# tensor([[1., 1., 1.],
#     [1., 1., 1.],
#     [1., 1., 1.],
#     [1., 1., 1.]]) torch.Size([4, 3])
# tensor([[1., 1., 1., 1., 1., 1.],
#     [1., 1., 1., 1., 1., 1.]]) torch.Size([2, 6])

t_stack = torch.stack([t,t,t], dim=0)
print(t_stack)
print(t_stack.shape)

t_stack1 = torch.stack([t, t, t], dim=1)
print(t_stack1)
print(t_stack1.shape)

# ## 结果：
# tensor([[[1., 1., 1.],
#          [1., 1., 1.]],

#         [[1., 1., 1.],
#          [1., 1., 1.]],

#         [[1., 1., 1.],
#          [1., 1., 1.]]])
# torch.Size([3, 2, 3])
# tensor([[[1., 1., 1.],
#          [1., 1., 1.],
#          [1., 1., 1.]],

#         [[1., 1., 1.],
#          [1., 1., 1.],
#          [1., 1., 1.]]])
# torch.Size([2, 3, 3])



**张量的拆分**

torch.chunk(input, chunks, dim=0): 将张量按维度dim进行平均切分， 返回值是张量列表，注意，如果不能整除， 最后一份张量小于其他张量。 chunks代表要切分的维度。

torch.split(tensor, split_size_or_sections, dim=0): 这个也是将张量按维度dim切分，但是这个更加强大， 可以指定切分的长度， split_size_or_sections为int时表示每一份的长度， 为list时，按list元素切分


**张量的索引**

torch.index_select(input, dim, index, out=None): 在维度dim上，按index索引数据，返回值，以index索引数据拼接的张量。

torch.masked_select(input, mask, out=None): 按mask中的True进行索引，返回值：一维张量。 input表示要索引的张量， mask表示与input同形状的布尔类型的张量。 这种情况在选择符合某些特定条件的元素的时候非常好使

**张量的变换**

torch.reshape(input, shape): 变换张量的形状，这个很常用,input表示要变换的张量，shape表示新张量的形状。 但注意，当张量在内存中是连续时， 新张量与input共享数据内存

torch.transpose(input, dim0, dim1): 交换张量的两个维度, 矩阵的转置常用， 在图像的预处理中常用， dim0要交换的维度， dim1表示要交换的维度

torch.t(input): 2维张量的转置， 对矩阵而言，相当于torch.transpose(inpuot, 0,1)


In [ ]:
# torch.reshape
t = torch.randperm(8)       # randperm是随机排列的一个函数
print(t)

t_reshape = torch.reshape(t, (-1, 2, 2))    # -1的话就是根据后面那两个参数，计算出-1这个值，然后再转
print("t:{}\nt_reshape:\n{}".format(t, t_reshape))

t[0] = 1024
print("t:{}\nt_reshape:\n{}".format(t, t_reshape))
print("t.data 内存地址:{}".format(id(t.data)))
print("t_reshape.data 内存地址:{}".format(id(t_reshape.data))) # 这个注意一下，两个是共内存的

## 结果：
# tensor([2, 4, 3, 1, 5, 6, 7, 0])
# t:tensor([2, 4, 3, 1, 5, 6, 7, 0])
# t_reshape:
# tensor([[[2, 4],
#          [3, 1]],

#         [[5, 6],
#          [7, 0]]])
# t:tensor([1024,    4,    3,    1,    5,    6,    7,    0])
# t_reshape:
# tensor([[[1024,    4],
#          [   3,    1]],

#         [[   5,    6],
#          [   7,    0]]])
# t.data 内存地址:1556953167336
# t_reshape.data 内存地址:1556953167336


## Tensor运算
Pytorch中提供了丰富的数学运算，可以分为三大类： 加减乘除， 对数指数幂函数，三角函数


torch.add(input, alpha=1, other, out=None): 逐元素计算input+alpha * other。 注意人家这里有个alpha，叫做乘项因子。类似权重的个东西。 这个东西让计算变得更加简洁， 比如线性回归我们知道有个y = wx + b， 在这里直接一行代码torch.add(b, w, x)就搞定。

类似的方法还有torch.addcdiv(input, value=1, tensor1, tensor2, out=None)，torch.addcmul(input, value=1, tensor1, tensor2, out=None)

In [ ]:
#统计值

a = torch.arange(1,10).float()
print(torch.sum(a))
print(torch.mean(a))
print(torch.max(a))
print(torch.min(a))
print(torch.prod(a)) #累乘
print(torch.std(a))  #标准差
print(torch.var(a))  #方差
print(torch.median(a)) #中位数

#cum扫描
a = torch.arange(1,10)

print(torch.cumsum(a,0))    # 累加
print(torch.cumprod(a,0))    # 累乘
print(torch.cummax(a,0).values)
print(torch.cummax(a,0).indices)
print(torch.cummin(a,0))

#torch.sort和torch.topk可以对张量排序
a = torch.tensor([[9,7,8],[1,3,2],[5,6,4]]).float()
print(torch.topk(a,2,dim = 0),"\n")
print(torch.topk(a,2,dim = 1),"\n")
print(torch.sort(a,dim = 1),"\n")

#利用torch.topk可以在Pytorch中实现KNN算法


矩阵运算包括：矩阵乘法，矩阵转置，矩阵逆，矩阵求迹，矩阵范数，矩阵行列式，矩阵求特征值，矩阵分解等运算。

In [ ]:
#矩阵乘法
a = torch.tensor([[1,2],[3,4]])
b = torch.tensor([[2,0],[0,2]])
print(a@b)  #等价于torch.matmul(a,b) 或 torch.mm(a,b)

#矩阵转置
a = torch.tensor([[1.0,2],[3,4]])
print(a.t())

#矩阵逆，必须为浮点类型
a = torch.tensor([[1.0,2],[3,4]])
print(torch.inverse(a))

#矩阵求trace
a = torch.tensor([[1.0,2],[3,4]])
print(torch.trace(a))

#矩阵求范数
a = torch.tensor([[1.0,2],[3,4]])
print(torch.norm(a))

#矩阵行列式
a = torch.tensor([[1.0,2],[3,4]])
print(torch.det(a))

#矩阵特征值和特征向量
a = torch.tensor([[1.0,2],[-5,4]],dtype = torch.float)
print(torch.eig(a,eigenvectors=True))

#两个特征值分别是 -2.5+2.7839j, 2.5-2.7839j 

#矩阵QR分解, 将一个方阵分解为一个正交矩阵q和上三角矩阵r
#QR分解实际上是对矩阵a实施Schmidt正交化得到q

a  = torch.tensor([[1.0,2.0],[3.0,4.0]])
q,r = torch.qr(a)
print(q,"\n")
print(r,"\n")
print(q@r)

#矩阵svd分解
#svd分解可以将任意一个矩阵分解为一个正交矩阵u,一个对角阵s和一个正交矩阵v.t()的乘积
#svd常用于矩阵压缩和降维
a=torch.tensor([[1.0,2.0],[3.0,4.0],[5.0,6.0]])

u,s,v = torch.svd(a)

print(u,"\n")
print(s,"\n")
print(v,"\n")

print(u@torch.diag(s)@v.t())

#利用svd分解可以在Pytorch中实现主成分分析降维




## 线性回归模型

In [4]:
x = torch.rand(20, 1) * 10
y = 2 * x + (5 + torch.randn(20, 1))

In [5]:
# 构建线性回归函数的参数
w = torch.randn((1), requires_grad=True)
b = torch.zeros((1), requires_grad=True)   # 这俩都需要求梯度

In [45]:
lr=0.01
for iteration in range(100):
	# 前向传播
	wx = torch.mul(w, x)
	y_pred = torch.add(wx, b)
	
	# 计算loss
	loss = (0.5 * (y-y_pred)**2).mean()
	
	# 反向传播
	loss.backward()
	
	# 更新参数

	b.data.sub_(lr * b.grad)    # 这种_的加法操作时从自身减，相当于-=
	w.data.sub_(lr * w.grad)

	# 梯度清零
	w.grad.data.zero_()
	b.grad.data.zero_()

print(w.data, b.data)


tensor([1.8901]) tensor([5.1778])
